In [1]:
from sklearn.model_selection import train_test_split
from biotransformers import BioTransformers
BioTransformers.list_backend()

/home/share/mambaforge/envs/biotf/lib/python3.7/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2023-10-20 11:49:18,242	INFO util.py:90 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.


Use backend in this list :

  *   esm1_t34_670M_UR100
  *   esm1_t12_85M_UR50S
  *   esm1_t6_43M_UR50S
  *   esm1b_t33_650M_UR50S
  *   esm_msa1_t12_100M_UR50S
  *   esm_msa1b_t12_100M_UR50S
  *   esm1v_t33_650M_UR90S_1
  *   protbert
  *   protbert_bfd


In [ ]:
import pandas as pd
data=pd.read_excel("./Production_similarity.xlsx")[['Protein','Sequence','Final product similarity']]
data=data.dropna()
X=data['Sequence'].to_numpy()
y=data['Final product similarity'].to_numpy()

bio_trans = BioTransformers("esm1_t6_43M_UR50S", num_gpus=1)
train_X,test_X,train_y,text_y=train_test_split(X,y)

In [ ]:
bio_trans.finetune(
    train_X,test_X,8,
    epochs=10,
    # checkpoint ='./check',
    accelerator = 'cpu',
    save_last_checkpoint=False
)

esm1_t6_43M_UR50S

In [ ]:
from biotransformers import BioTransformers
# 多GPU需要导入
# import ray
# ray.init()

sequences = data['Sequence'].to_list()

bio_trans = BioTransformers("esm1_t6_43M_UR50S",num_gpus=1)
# bio_trans = BioTransformers("esm1b_t33_650M_UR50S",num_gpus=1)
embeddings = bio_trans.compute_embeddings(sequences, pool_mode=("mean"), batch_size=8)

# pool_mode
# cls : return the <CLS> token embedding in the sequence.
# mean : if sequence has shape (num_token, embedding_size), the num_token dimension is averaging and the embedding has shape (num_token,)
# full : no pooling function applied, all the embeddings for each sequence are return.

embeddings

Pseudo-Loglikelihood

In [ ]:
from biotransformers import BioTransformers

sequences = [
        "MKTVRQERLKSIVRILERSKEPVSGAQLAEELSVSRQVIVQDIAYLRSLGYNIVATPRGYVLAGG",
        "KALTARQQEVFDLIRDHISQTGMPPTRAEIAQRLGFRSPNAAEEHLKALARKGVIEIVSGASRGIRLLQEE",
    ]

bio_trans = BioTransformers(backend="esm1_t6_43M_UR50S",num_gpus=0)
loglikelihood = bio_trans.compute_loglikelihood(sequences)
loglikelihood

## compute_mutation_score function

To compute this metric, we need to provide to things:

sequences: a list of sequence to score

mutations: a list of sequence’ mutations. A sequence” mutations can be composed of multiple single mutations.

In [ ]:
from biotransformers import BioTransformers
bio_trans = BioTransformers("esm1_t6_43M_UR50S",num_gpus=0)

sequences = ["MAPSRKFFVGGNWKMNVVCAPPTAYIDFARQKLDPKI",
             "AVAAQNCYKVTNGAFTGEISPGMIKDCGATWVVLGH",
             "GRKQSLGELIGTLNAAKVPADTE"]

mutations = [["M1P","K6F","N12G"],["A4V"],["A15L","V18A"]]
bio_trans.compute_mutation_score(sequences,mutations)

pdb

In [ ]:
!curl -X POST --data "MKTVRQERLKSIVRILERSKEPVSGAQLAEELSVSRQVIVQDIAYLRSLGYNIVATPRGYVLAGG" https://api.esmatlas.com/foldSequence/v1/pdb/ > test.pdb
